In [100]:
import requests
from BeautifulSoup import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [112]:
url = "http://www.espncricinfo.com/ci/engine/match/index/live.html"
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)

In [102]:
for i in soup.findAll('h2'):
    print i.text
    print 

One-Day Internationals

Twenty20 Internationals

Women's One-Day Internationals

Ireland A in Bangladesh unofficial Test Match

Logan Cup

National Cricket League

Sri Lanka A in West Indies unofficial Test series

Sunfoil 3-Day Cup

ICC World Cricket League Championship

JLT One-Day Cup

New Zealand A in India unofficial ODI Series



In [139]:
def innings_parser(x, inning=1):
    if '&amp;' in x:
        return re.split(r'(^[^\d]+)', x)[2].split(' &amp; ')[inning-1]
    elif inning > 1:
        return np.nan
    else:
        return re.split(r'(^[^\d]+)', x)[2]
    
def get_format(x):
    if 'unofficial' not in x :
        if 'Test' in x:
            return 'Test'
        elif 'ODI' in x:
            return 'ODI'
        elif 'T20I' in x:
            return 'T20I'
        else:
            return 'other'
    else:
        return 'other'

In [138]:
details = {
'match_details': [],
'match_link': [],
'match_status': [],
'team1': [],
'team2': []
}

for i in soup.findAll('span', attrs={'class': 'match-no'}):
    details['match_details'].append(i.text)
    details['match_link'].append(i.a['href'])

for i in soup.findAll('div', attrs={'class': 'match-status'}):
    details['match_status'].append(i.text)

for i in soup.findAll('div', attrs={'class': 'innings-info-1'}):
    details['team1'].append(i.text)
    
for i in soup.findAll('div', attrs={'class': 'innings-info-2'}):
    details['team2'].append(i.text)
    
df = pd.DataFrame(details)
df['batting_first_team'] = df.team1.apply(lambda x: re.split(r'(^[^\d]+)', x)[1])
df['batting_second_team'] = df.team2.apply(lambda x: re.split(r'(^[^\d]+)', x)[1])
df['innings1_score'] = df.team1.apply(innings_parser, args=(1,))
df['innings2_score'] = df.team2.apply(innings_parser, args=(1,))
df['innings3_score'] = df.team1.apply(innings_parser, args=(2,))
df['innings4_score'] = df.team2.apply(innings_parser, args=(2,))
df['match_status'] = df.match_status.apply(lambda x: x.replace('&nbsp;', ' '))
df['match_format'] = df.match_details.apply(get_format)
df['match_format'] = pd.Categorical(df['match_format'], ["Test", "ODI", "T20I", 'other'])
df = df.sort_values('match_format').reset_index(drop=True)
df.head()

,match_details,match_link,match_status,team1,team2,batting_first_team,batting_second_team,innings1_score,innings2_score,innings3_score,innings4_score,match_format
0,1st ODI at Dubai International Cricket Stadium...,http://www.espncricinfo.com/series/18037/game/...,Sri Lanka require another 249 runs with 8 wick...,Pakistan292/6 (50 ov),Sri Lanka44/2 (8/50 ov),Pakistan,Sri Lanka,292/6 (50 ov),44/2 (8/50 ov),NaN,NaN,ODI
1,"2nd ODI at Brian Lara Stadium, Tarouba, Trinid...",http://www.espncricinfo.com/series/8674/game/1...,Match scheduled to begin in 2 hours 7 mins(14:...,West Indies Women,Sri Lanka Women,West Indies Women,Sri Lanka Women,,,NaN,NaN,ODI
2,3rd T20I at Rajiv Gandhi International Stadium...,http://www.espncricinfo.com/series/17974/game/...,Match abandoned without a ball bowled,India,Australia,India,Australia,,,NaN,NaN,T20I
3,"16th match at Hurstville Oval, Sydney",http://www.espncricinfo.com/series/8626/scorec...,Tasmania won by 114 runs,Tasmania334/7 (50 ov),Cricket Australia XI220 (46.2/50 ov),Tasmania,Cricket Australia XI,334/7 (50 ov),220 (46.2/50 ov),NaN,NaN,domestic
4,"47th Match at Mission Road Ground, Mong Kok, H...",http://www.espncricinfo.com/series/8650/game/1...,Hong Kong won by 83 runs,Hong Kong194 (50 ov),Nepal111 (34.5/50 ov),Hong Kong,Nepal,194 (50 ov),111 (34.5/50 ov),NaN,NaN,domestic


In [131]:
for i in df.itertuples():
    print i.match_format
    print '{} v {} - {}'.format(i.batting_first_team, i.batting_second_team, i.match_details)
    print '{}'.format(i.match_status)
    print ''

ODI
Pakistan v Sri Lanka - 1st ODI at Dubai International Cricket Stadium (day/night)
Sri Lanka require another 249 runs with 8 wickets and 42.0 overs remaining

T20I
India v Australia - 3rd T20I at Rajiv Gandhi International Stadium, Uppal, Hyderabad (night)
Match abandoned without a ball bowled

ODI
West Indies Women v Sri Lanka Women - 2nd ODI at Brian Lara Stadium, Tarouba, Trinidad (day/night)
Match scheduled to begin in 2 hours 7 mins(14:00 local | 18:00 GMT)

domestic
Ireland A v Bangladesh A - Only unofficial Test at Sylhet International Cricket Stadium
Stumps - Bangladesh A require another 105 runs with 8 wickets remaining

domestic
Mountaineers v Mid West Rhinos - at Harare Sports Club
Mountaineers won by 2 runs

domestic
Rangpur Division v Dhaka Division - Tier 1 at Sheikh Abu Naser Stadium, Khulna
Stumps - Rangpur Division won the toss and elected to bat

domestic
Sylhet Division v Dhaka Metropolis - Tier 2 at Zahur Ahmed Chowdhury Stadium, Chittagong
Stumps - Sylhet Divisi

In [108]:
for url in df.match_link:
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    print soup.find('title').text.split(' |')[0]

SL 35/2  (7.0 ov, WU Tharanga 10*, HDRL Thirimanne 0*, Junaid Khan 0/19) - Live
3rd T20I (N), Australia tour of India at Hyderabad, Oct 13 2017
2nd ODI (D/N), ICC Women's Championship at Tarouba, Oct 13 2017
BdshA 27/2  (11.0 ov, Shadman Islam 14*, Al-Amin 9*, GH Dockrell 0/9) - Stumps
Logan Cup at Harare, Oct 10-13 2017
RgDiv 321/4  (89.2 ov, Naeem Islam 120*, Suhrawadi Shuvo 145*, Mohammad Sharif 0/42) - Stumps
SyDiv 241/6  (90.0 ov, Ezaz Ahmed 27*, Rahatul Ferdous 1*, Nihaduzzaman 2/69) - Stumps
KhDiv 307/4  (90.0 ov, Ziaur Rahman 52*, Nahidul Islam 11*, Tawhidul Islam 0/42) - Stumps
RaDiv 34/2  (9.0 ov, Mizanur Rahman 20*, Delwar Hossain 0*, Hasan Mahmud 0/7) - Stumps
WI A 364/8  (120.0 ov, SS Cottrell 2*, K Joseph 1*, CBRLS Kumara 0/63) - Stumps
EPR 36/3  (11.0 ov, KR Smuts 6*, T Koekemoer 6*, TS Motaung 0/7) - Stumps
EASTN 101/7  (29.0 ov, M Arnold 17*, CJ August 5*, S Nhlebela 1/9) - Live
NWEST 330/5  (79.0 ov, BL Barends 12*, K Rapulana 37*, S Jamison 2/58) - Match delayed - Ba

In [159]:
from os.path import expanduser

with open(os.path.expanduser('~/.bash_profile'), 'r') as f:
    data = f.read()

In [160]:
data

'export SPARK_HOME=/Users/khani26/Desktop/software/spark-2.2.0-bin-hadoop2.7\n\nexport JAVA_OPTS="-Xms512m -Xmx2g -Xss256M -XX:MaxPermSize=2048m -XX:+CMSClassUnloadingEnabled -XX:ReservedCodeCacheSize=512m -Dhttp.proxyHost=vfukukproxy.internal.vodafone.com\n-Dhttp.proxyPort=8080 -Dhttp.sslVerify=false"\nexport SBT_OPTS="$JAVA_OPTS $SBT_OPTS"\nexport USER=khani26\nexport BDA_USER_HOME=~/workspace/bda-core \nexport PATH=$PATH:$SCALA_HOME/bin:$SBT_HOME/bin\nexport PYTHONPATH="$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.9-src.zip:/Users/khani26/anaconda/bin:/Users/khani26/Documents/bda-core:/Users/khani26/Documents/" \nexport SBT_HOME=/usr/local/Cellar/sbt\nexport SCALA_HOME="/Users/khani26/Desktop/software/scala-2.10.4"\n# added by Anaconda2 4.3.0 installer\nexport PATH="/Users/khani26/anaconda/bin:$PATH"\nexport PATH="$PATH:$SCALA_HOME/bin"\n\nexport PATH="/Users/khani26/anaconda/bin:$PATH"\nexport PATH=/Users/khani26/Desktop/software/apache-maven-3.5.0/bin:$PATH\n\n# Enable tab com